In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
df_raw = pd.read_csv('./data.csv')
df_raw.rename(columns = {'Sentence':'sentence', 'Sentiment':'sentiment'}, inplace = True)

In [3]:
df_raw.head()

,sentence,sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [4]:
print(len(df_raw))

5842


In [5]:
df_raw.sentiment.values
set(df_raw.sentiment.values)

{'negative', 'neutral', 'positive'}

In [6]:
df = df_raw[df_raw.sentiment != "neutral"] # only take rows that does not have 'neutral' in the choose_one column

In [7]:
df.shape

(2712, 2)

In [8]:
sentiment_value = dict({
                    'positive':1,
                    'negative':0
                  })
#Dictionary 

In [9]:
df['value'] = df.sentiment.map(sentiment_value)

<ipython-input-9-6633665d2415>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['value'] = df.sentiment.map(sentiment_value)


In [10]:
df.head()

,sentence,sentiment,value
0,The GeoSolutions technology will leverage Bene...,positive,1
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,0
2,"For the last quarter of 2010 , Componenta 's n...",positive,1
5,$SPY wouldn't be surprised to see a green close,positive,1
6,Shell's $70 Billion BG Deal Meets Shareholder ...,negative,0


In [11]:
#Making of hash map for bag of words /term frequency(tf)

def make_hash_map(df): #Main function
    hash_map = {}
    for index, row in df.iterrows():
        hash_map = map_book(hash_map, extract_words(row['sentence']))
    return hash_map

#This is a function that has been defined for tokenization of words
def extract_words(sentence):
    '''This is to clean and tokenize words'''
    ignore_words = ['a', 'the', 'if', 'br', 'and', 'of', 'to', 'is', 'are', 'he', 'she', 'my', 'you', 'it','how']
    words = re.sub("[^\w]", " ",  sentence).split() # this replaces all special chars with ' '
    words = [word.lower() for word in words]
    words_cleaned = [w.lower() for w in words if w not in ignore_words]
    return words_cleaned 

def map_book(hash_map, tokens):
    if tokens is not None:
        for word in tokens:
            # Word Exist?
            if word in hash_map:
                hash_map[word] = hash_map[word] + 1
            else:
                hash_map[word] = 1

        return hash_map
    else:
        return None

In [12]:
# define a function frequent_vocab with the following input: word_freq and max_features
#Not understood !!
def frequent_vocab(word_freq, max_features): 
    counter = 0  #initialize counter with the value zero
    vocab = []   # create an empty list called vocab
    # list words in the dictionary in descending order of frequency
    for key, value in sorted(word_freq.items(), key=lambda item: (item[1], item[0]), reverse=True): 
       #loop function to get the top (max_features) number of words
        if counter<max_features: 
            vocab.append(key)
            counter+=1
        else: break
    return vocab

In [13]:
hash_map = make_hash_map(df)
#experiment with vocab part 500 not fixed
vocab=frequent_vocab(hash_map, 500)

In [14]:
vocab

['in',
 'eur',
 'for',
 'from',
 's',
 'mn',
 'on',
 'profit',
 '1',
 'sales',
 'year',
 'its',
 'by',
 'net',
 'company',
 '2',
 'finnish',
 'co',
 'with',
 'said',
 'has',
 '3',
 'million',
 '5',
 'as',
 'will',
 'm',
 'period',
 'at',
 '4',
 'up',
 'operating',
 'quarter',
 '0',
 'mln',
 '6',
 'that',
 '2009',
 '7',
 'was',
 '8',
 '2008',
 '9',
 't',
 'be',
 'loss',
 '2007',
 'this',
 'first',
 '2010',
 'http',
 'share',
 'compared',
 'oyj',
 'group',
 'stks',
 'https',
 'down',
 'today',
 'an',
 'new',
 'corresponding',
 'finland',
 'euro',
 'market',
 'increased',
 'rose',
 'percent',
 '2006',
 'long',
 'have',
 '000',
 'aapl',
 '10',
 'price',
 'also',
 'after',
 'per',
 'business',
 '2005',
 'which',
 'than',
 'increase',
 'been',
 'we',
 'i',
 'hel',
 'contract',
 'our',
 'last',
 'buy',
 'third',
 'more',
 'earlier',
 'operations',
 'growth',
 'earnings',
 'over',
 'decreased',
 'tsla',
 'same',
 'services',
 'pct',
 'helsinki',
 'shares',
 'second',
 'now',
 'result',
 'stock

In [15]:
# define a function bag of words with the following input: sentence and words
def bagofwords(sentence, words):
    sentence_words = extract_words(sentence) #tokenize sentences and assign it to variable sentence_words
    # frequency word count
    bag = np.zeros(len(words)) #create a NumPy array made up of zeroes with size len(words)
    # loop through data and add value of 1 when token is present in the tweet
    for sw in sentence_words:
        for i,word in enumerate(words):
            if word == sw: 
                bag[i] += 1
                
    return np.array(bag) # return the bag of word for one tweet

In [16]:
no_words = len(vocab)
no_documents = len(df)
bag_ow = np.zeros([no_documents ,no_words])
for i in range(no_documents) :
    bag_ow[i ,:] = bagofwords(df['sentence'].iloc[i],vocab)

In [17]:
bag_ow.shape

(2712, 500)

In [18]:
bag_ow

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [19]:
sorted(hash_map.items(), key=lambda item: item[1], reverse=True)[:20]

[('in', 1391),
 ('eur', 748),
 ('for', 619),
 ('from', 588),
 ('s', 499),
 ('mn', 459),
 ('on', 385),
 ('profit', 357),
 ('1', 322),
 ('sales', 321),
 ('year', 318),
 ('its', 305),
 ('by', 300),
 ('net', 297),
 ('company', 283),
 ('2', 275),
 ('finnish', 272),
 ('co', 268),
 ('with', 262),
 ('said', 258)]

In [20]:
numdocs, numwords = np.shape(bag_ow)
N = numdocs
word_frequency = np.empty(numwords)
word_frequency

array([0.00000000e+000, 6.93745691e-310, 0.00000000e+000, 6.93745691e-310,
       6.93745691e-310, 6.93744084e-310, 0.00000000e+000, 3.95252517e-323,
       6.93744084e-310, 6.93745694e-310, 6.93745694e-310, 6.93745694e-310,
       6.93745694e-310, 6.93745694e-310, 6.93745695e-310, 6.93745695e-310,
       6.93745695e-310, 6.93745695e-310, 6.93745695e-310, 6.93745695e-310,
       6.93745695e-310, 6.93745695e-310, 6.93745695e-310, 6.93745695e-310,
       6.93745695e-310, 6.93745695e-310, 6.93745695e-310, 6.93745695e-310,
       6.93745695e-310, 6.93745694e-310, 6.93745694e-310, 6.93745694e-310,
       6.93745694e-310, 6.93745694e-310, 6.93745694e-310, 6.93745694e-310,
       6.93745694e-310, 6.93745694e-310, 6.93745694e-310, 6.93745694e-310,
       6.93745694e-310, 6.93745695e-310, 6.93745695e-310, 6.93745695e-310,
       6.93745695e-310, 6.93745695e-310, 6.93745695e-310, 6.93745694e-310,
       6.93745694e-310, 6.93745694e-310, 6.93745694e-310, 6.93745694e-310,
       6.93745695e-310, 6

In [21]:
numwords

500

In [22]:
for word in range(numwords):
    word_frequency[word] = np.sum((bag_ow[:,word]>0))
idf = np.log(N/word_frequency)

In [23]:
idf.shape

(500,)

In [24]:
#initializs tfidf array
tfidf = np.empty([numdocs, numwords])

#loop through the tweets, multiply term frequency (represented by bag of words) with idf
for doc in range(numdocs):
    tfidf[doc, :]=bag_ow[doc, :]*idf

In [25]:
tfidf.shape

(2712, 500)

In [26]:
tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 3.23888711, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9917043 , 0.        , 1.61944355, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.61944355, ..., 0.        , 0.        ,
        0.        ]])

In [27]:
from sklearn.linear_model import LogisticRegression #to import logistic regression model
from sklearn.model_selection import train_test_split #to split data into training and testing set
from sklearn.model_selection import GridSearchCV #to find out the best parameter for our model

In [31]:
# split X_all and y_all into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(tfidf,df['sentiment'].values,shuffle=True)

In [33]:
print(X_train.shape)
print(X_test.shape)

print(y_train.shape)
print(y_test.shape)

(2034, 500)
(678, 500)
(2034,)
(678,)


In [34]:
# Create a model instance
logreg = LogisticRegression(solver = 'lbfgs')

In [35]:
#Fit the model on the training set
logreg.fit(X_train,y_train)

LogisticRegression()

In [36]:
y_pred=logreg.predict(X_test)
print (y_pred)

['negative' 'positive' 'positive' 'negative' 'negative' 'positive'
 'positive' 'negative' 'positive' 'positive' 'negative' 'positive'
 'positive' 'negative' 'negative' 'positive' 'positive' 'positive'
 'positive' 'negative' 'positive' 'negative' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'negative'
 'negative' 'positive' 'positive' 'positive' 'positive' 'positive'
 'negative' 'positive' 'positive' 'positive' 'positive' 'positive'
 'negative' 'positive' 'positive' 'negative' 'negative' 'positive'
 'negative' 'positive' 'positive' 'negative' 'negative' 'positive'
 'positive' 'positive' 'positive' 'negative' 'positive' 'negative'
 'positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'negative' 'positive'
 'negative' 'positive' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'negative' 'positive' 'positive'
 'negative' 'positive' 'positive' 'positive' 'positive' 'posit

In [37]:
score = logreg.score(X_test, y_test)
print(score)

0.7905604719764012


In [68]:
def sentiment_predictor(news):
    # your code here
    word_vector = bagofwords(news, vocab) # set a variable with bag of words. Remember the bagofwords function you have created?
    word_tfidf = word_vector*idf #find tfidf value
    prediction = logreg.predict(word_tfidf.reshape(1, -1)) #predict wether news is positive or not
    result = {'positive' :1, 'negative' :2}
    #results = {1:'positive', 0:'negative'} #creating a set containing the potential results. 
    #print(result[int(prediction)])
    print(str(prediction))

In [75]:
sentence_1 = "Operating profit fell to USD 38.1 mn from USD 40" #negative
sentence_2 = "ABC wouldn't be surprised to see a green close" #positive
sentiment_predictor(sentence_2)

['positive']


In [60]:
df

,sentence,sentiment,value
0,The GeoSolutions technology will leverage Bene...,positive,1
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,0
2,"For the last quarter of 2010 , Componenta 's n...",positive,1
5,$SPY wouldn't be surprised to see a green close,positive,1
6,Shell's $70 Billion BG Deal Meets Shareholder ...,negative,0
...,...,...,...
5832,Operating profit fell to EUR 38.1 mn from EUR ...,negative,0
5835,HSBC Says Unit to Book $585 Million Charge on ...,negative,0
5836,Daily Mail parent company in talks with potent...,positive,1
5837,RISING costs have forced packaging producer Hu...,negative,0


In [77]:
sentence = input('What is the news ?')
sentiment_predictor(sentence)

What is the news ?stocks fell by 33%
['negative']
